In [1]:
print("Hello World!")

Hello World!


In [46]:
from langchain_community.llms import Ollama
REMOTE="https://6c56-34-32-145-91.ngrok-free.app"
MODEL="llama3"


In [47]:
model = Ollama(base_url=REMOTE,model=MODEL, temperature=0)

model.invoke("Hello World!")

'Hello there! It\'s great to meet you! "Hello World" is a classic greeting, and I\'m happy to be on the receiving end of it. How can I help or chat with you today?'

In [48]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = model | parser

chain.invoke("Hello World!")

'Hello there! It\'s great to meet you! "Hello World" is a classic greeting, and I\'m happy to be on the receiving end of it. How can I help or chat with you today?'

In [32]:
# One Document


from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/pdfs/main doc.pdf")
pages = loader.load_and_split()
pages


[Document(metadata={'source': 'data/pdfs/main doc.pdf', 'page': 0}, page_content='main doc.md 2024-09-18\n1 / 5LightSpeedPay - Payment Gateway Documentation\nTable of Contents\n1. Introduction\n2. Technology Stack\n3. Folder Structure\n4. Conﬁguration\n5. Running the Application\n6. API Routes\n7. Middleware\n8. Environment Variables\n9. Testing\n10. Database Setup\n11. Seeding Data\n12. Error Handling\n13. Important Notes\nIntroduction\nLightSpeedPay is a payment gateway that allows merchants to process payments eﬃciently. This\ndocumentation provides an overview of the system architecture, the key components, and instructions for\nsetting up and maintaining the system.\nThe application is built using Node.js and Express.js for the server-side logic, and MongoDB as the\ndatabase. It includes various RESTful APIs for managing merchants, transactions, wallets, banks, and\npricing.\nTechnology Stack\nNode.js (v16.x+)\nExpress.js (v4.x)\nMongoDB (v6.x) - for storing payment data and merch

In [49]:
# All PDFs in the folder

import os
from langchain_community.document_loaders import PyPDFLoader

# Assuming the directory where all PDFs are stored is "data/pdfs"
pdf_folder = "data/pdfs"

# List to store all the loaded pages
all_pages = []

# Iterate over all files in the folder
for filename in os.listdir(pdf_folder):
    if filename.endswith(".pdf"):  # Ensure only PDF files are loaded
        filepath = os.path.join(pdf_folder, filename)
        loader = PyPDFLoader(filepath)
        pages = loader.load_and_split()  # Load and chunk each PDF
        all_pages.extend(pages)  # Append pages to the main list

# Now `all_pages` contains all the chunked pages from all PDFs in the folder
pages = all_pages
pages

[Document(metadata={'source': 'data/pdfs/authmiddleware.pdf', 'page': 0}, page_content='authmiddleware.md 2024-09-18\n1 / 5Authentication and Authorization Middleware\nDocumentation\nThis documentation covers the implementation of a middleware that manages authentication and\nauthorization in a Node.js application using JWT (JSON Web Tokens). The middleware ensures that\nprotected routes are only accessible to authenticated users and performs permission-based access control\nfor speciﬁc resources.\nTable of Contents\n1. Dependencies\n2. protect Middleware\nSkipped URLs\nRestricted URLs\nToken Veriﬁcation\nMerchant ID Veriﬁcation\n3. admin Middleware\n4. authenticateToken Middleware\n5. allowIfPermission Middleware\n6. mapUrlToPermissions Function\nDependencies\nconst jwt = require("jsonwebtoken" ); \nconst User = require("../models/merchantModel" ); \nconst AffiliateUser = require("../models/affiliateUsersModels" ); \nconst dotenv = require(\'dotenv\' ); \njsonwebtoken : Used to verify

In [51]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(template=template)
print(prompt.format(context="context", question="question"))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

context

Question: question
Helpful Answer:


In [52]:
chain = prompt | model | parser

In [53]:
chain.invoke({
    "context": "My name is Rishabh and I am not so good at study",
    "question": "Who is RIshabh?"
})

'According to the context, Rishabh is the person whose name is mentioned.'

In [61]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
# Instantiate the embedding object
embedding = OllamaEmbeddings(base_url=REMOTE,model=MODEL)

# Embed the documents
# embedded_documents = embedding.embed_documents(pages)

# Create the vectorstore with the embedded documents
# vectorstore = DocArrayInMemorySearch.from_documents(pages,embedding=embedding)
vectorstore = Chroma.from_documents(pages,collection_name="maindoc", embedding=embedding,persist_directory="db")

In [59]:
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("seeder.js")

[Document(metadata={'page': 1, 'source': 'data/pdfs/transaction controller.pdf'}, page_content="transaction controller.md 2024-09-18\n2 / 3Updates the status of a transaction (success or failed) based on user input.\nSends a callback with the updated status.\n6. deleteAllTransaction :\nDeletes all transaction records in the database.\n7. getTransactionStats :\nAggregates transaction statistics, including today's and this month's transaction volumes and\nproﬁts.\nAvailable for both admin and non-admin merchants.\n8. progressBar :\nFetches progress bar data based on completed or successful transactions.\nAdmins can view all transactions, while merchants only see their own.\n9. getTransactionStatus :\nReturns the current status of a speciﬁc transaction.\n10. updateTransactionStatusICICI :\nUpdates the transaction status in ICICI's system.\n11. collectRequest :\nInitiates a collect request for a UPI transaction.\nVeriﬁes the merchant's wallet balance and status before proceeding.\n12. requ

In [60]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")} | prompt | model | parser)

chain.invoke({"question": "What is LightspeedPay?"})

"I don't know. The provided context does not explicitly mention what LightspeedPay is, but it appears to be a payment gateway or system that handles transactions and provides APIs for managing payments."

In [62]:
question = [
    "What is LightspeedPay?",
    "How do I use LightspeedPay?",
    "What is the difference between LightspeedPay and other payment gateways?",
    "What is the techstack of LightspeedPay?",
]

for q in question:
    print(f"Question: {q}")
    print(f"Answer: {chain.invoke({'question': q})}")
    print("\n")

Question: What is LightspeedPay?
Answer: I don't know. The provided context does not explicitly mention what LightspeedPay is, but it appears to be a payment gateway or system that handles transactions and provides APIs for managing payments.


Question: How do I use LightspeedPay?
Answer: Based on the provided context, here is a helpful answer:

To use LightspeedPay, you can follow these steps:

1. Initiate a transaction by making an API call with the required parameters, such as customer name, status, method, description, amount, bill ID, VPA ID, API key, and API secret.
2. The API response will include a payment link that should be presented to the customer.
3. The customer completes the payment by being redirected to the LightSpeedPay payment page via the payment link.
4. Ensure that the browser or WebView supports JavaScript and DOM storage for the payment page to function properly.
5. Based on the outcome of the payment, the customer is redirected to either the success or fail UR